In [1]:
!pip install surprise

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from surprise import Reader, Dataset, SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise.accuracy import rmse

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

In [3]:
users = pd.read_csv('../input/users.csv')
posts = pd.read_csv('../input/posts.csv')
views = pd.read_csv('../input/views.csv')

In [4]:
users.rename(columns = {'_id':'user_id'},inplace=True)
# print(users.academics.unique())
# print(users.gender.unique())
# print(users.describe())
users.head()

,user_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [5]:
posts.rename(columns = {'_id':'post_id',' post_type':'post_type'},inplace=True)
posts.category = posts.category.fillna('')
# print(posts.post_type.unique())
# print(posts.describe())
posts.head()

,post_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [6]:
# print(views.describe())
views.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [7]:
df = pd.merge(views,users,on='user_id')
df = pd.merge(df,posts,on='post_id')
# df.drop(columns=['name','timestamp'],inplace = True)
df.head()

,user_id,post_id,timestamp,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,2020-05-22T20:11:32.317Z,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,2020-05-22T20:01:37.309Z,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,2020-05-22T20:10:41.100Z,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,2020-05-22T20:08:45.061Z,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


# PROBLEM 2  
# -> Recommend similar posts for the given post

In [8]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf.fit_transform(posts['title'])
tfidf_matrix2 = tf.fit_transform(posts['category'])
tfidf_matrix3 = tf.fit_transform(posts['post_type'])
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)
cosine_sim3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)
cosine_sim_Tfidf = (cosine_sim1 + cosine_sim2 + cosine_sim3)/3

In [9]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
def create_mixture(x):
    return x['title1'] + ' ' + x['category1'] + ' ' + x['post_type1']

features = ['title', 'category', 'post_type']
for feature in features:
    posts[feature+'1'] = posts[feature].apply(clean_data)     
posts['mixture'] = posts.apply(create_mixture, axis=1)

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(posts['mixture'])
cosine_sim_count = cosine_similarity(count_matrix, count_matrix)

posts.drop(columns = ['title1','category1','post_type1','mixture'],inplace=True)

In [10]:
indices = pd.Series(posts.index, index=posts.title)
def get_recommendations(post,cosine_sim):
    idx = indices[post]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    post_indices = [i[0] for i in sim_scores]
    return posts.loc[post_indices]

In [11]:
get_recommendations('Ml and AI',cosine_sim_Tfidf).head()

,post_id,title,category,post_type
170,5e7c78fdcfc8b713f5ac7daa,Types Of AI.,Computer Technology|Artificial Intelligence,blog
224,5e897ab4a3258347b42f25ca,10 Best Artificial Intelligence (AI) Courses O...,Computer Technology|Machine Learning,blog
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
186,5e81a4f2a3258347b42f21d3,7 Best Python Data Science Courses & Certifica...,Computer Technology|Machine Learning,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog


In [12]:
get_recommendations('Ml and AI',cosine_sim_count).head()

,post_id,title,category,post_type
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
169,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog
291,5e966ad3a3258347b42f2aaf,8 Best Machine Learning Courses for 2020,Computer Technology|Cloud Computing|Artificial...,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog
16,5dadf6ae610ba040fbfdf5c8,MIS,Computer Technology|Information Technology,blog


In [13]:
get_recommendations('Ml and AI',(cosine_sim_count + cosine_sim_Tfidf)/2).head()

,post_id,title,category,post_type
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
170,5e7c78fdcfc8b713f5ac7daa,Types Of AI.,Computer Technology|Artificial Intelligence,blog
169,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog
291,5e966ad3a3258347b42f2aaf,8 Best Machine Learning Courses for 2020,Computer Technology|Cloud Computing|Artificial...,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog


# PROBLEM 1  
# -> Recommend posts for the given user

In [14]:
f1 = {'artwork': 1, 
      'blog'  : 2, 
      'project' : 3, 
      'skill' : 4}

f2 = {'female' : 7.4, 
      'male' : 6.2, 
      'undefined' : 5.1}

f3 = {'undergraduate' : 9.99, 
      'graduate' : 9.23, 
      'undefined' : 8.05}

df['strength'] = ( (df['post_type'].apply(lambda x: f1[x]))/4 + (df['gender'].apply(lambda x: f2[x]))/7.4 + (df['academics'].apply(lambda x: f3[x]))/9.99 )/3
df['strength'] = 4.99*(df['strength'].values/max(df['strength'].values))

df = df[['user_id','post_id','strength']]
df.head()

,user_id,post_id,strength
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,3.74250
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,3.47277
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,3.61596
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,3.34623
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,3.47277


In [15]:
# pivot_matrix = pd.pivot_table(df,index='user_id', columns='post_id', values='strength')
# pivot_matrix = pivot_matrix.fillna(pivot_matrix.mean(axis=0))
# pivot_matrix.head()

In [16]:
reader = Reader()
data = Dataset.load_from_df(df, reader)
benchmark = []
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=5, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

In [17]:
results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
results

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNNBaseline,0.166287,0.112967,0.005172,0.009294
KNNWithMeans,0.197010,0.138974,0.003996,0.008089
KNNBasic,0.210835,0.158060,0.001514,0.007327
KNNWithZScore,0.218795,0.146644,0.008994,0.007962
SlopeOne,0.236506,0.172621,0.013583,0.020503
SVDpp,0.254370,0.192050,1.862957,0.031116
BaselineOnly,0.293849,0.226406,0.003593,0.002488
SVD,0.294185,0.224815,0.134884,0.004316
NMF,0.331440,0.246103,0.213357,0.003358


In [18]:
# Using Alternating Least Squares (ALS)

sim_options = {
    "name": 'pearson_baseline' ,
    "user_based": True,
    'shrinkage': 0
}

# ['pearson_baseline','msd', 'cosine']
# ['als', 'sgd'],

bsl_options = {'method': 'als', 
#                'learning_rate': .00005,
#                'reg': 2,
#                'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

algo = KNNBaseline(sim_options=sim_options, bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.14838845, 0.16525484, 0.17444093, 0.15956375, 0.16489882]),
 'test_mae': array([0.09802697, 0.10571255, 0.11390314, 0.10126198, 0.10254677]),
 'fit_time': (0.006136178970336914,
  0.0063359737396240234,
  0.005602359771728516,
  0.0056307315826416016,
  0.005789279937744141),
 'test_time': (0.008070230484008789,
  0.0075910091400146484,
  0.0076100826263427734,
  0.0076389312744140625,
  0.008055925369262695)}

In [19]:
trainset, testset = train_test_split(data, test_size=0.2)
algo = KNNBaseline(algo = KNNBaseline(sim_options=sim_options, bsl_options=bsl_options))
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1731


0.17309805427450767

In [20]:
algo.__class__.__name__

'KNNBaseline'

In [21]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df_new = pd.DataFrame(predictions, columns=['user_id', 'post_id', 'rui', 'est', 'details'])
df_new['no_item_rated_by_user'] = df_new.user_id.apply(get_Iu)
df_new['no_user_rated_item'] = df_new.post_id.apply(get_Ui)
df_new['error'] = abs(df_new.est - df_new.rui)
df_new.head()

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
0,5d6d2bb87fa40e1417a49315,5ecc1d4eeaff6b0c3a58a565,3.472770,3.400358,"{'actual_k': 10, 'was_impossible': False}",23,10,0.072412
1,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,3.472770,3.472770,"{'actual_k': 1, 'was_impossible': False}",191,1,0.000000
2,5e26bb482a37d20505da2c10,5e2867f6025ae20886de205a,3.888604,3.835876,{'was_impossible': False},3,0,0.052728
3,5e5af599d701ab08af792b63,5ec7a8bdec493f4a26558846,3.615960,3.667104,"{'actual_k': 9, 'was_impossible': False}",50,10,0.051144
4,5d60098a653a331687083238,5eac350110426255a7aa9e92,3.888604,3.891741,"{'actual_k': 4, 'was_impossible': False}",191,4,0.003138


In [22]:
best_predictions = df_new.sort_values(by='error')[:5]
worst_predictions = df_new.sort_values(by='error')[-5:]

In [23]:
best_predictions

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
1,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,3.472770,3.472770,"{'actual_k': 1, 'was_impossible': False}",191,1,0.000000
112,5df20f1fee4bb5252b4f5351,5e5196b5d701ab08af7929a8,3.888604,3.888604,"{'actual_k': 1, 'was_impossible': False}",60,1,0.000000
274,5deeef6142a8854bf6eabab9,5e947c4aa3258347b42f283e,3.888604,3.888604,"{'actual_k': 1, 'was_impossible': False}",58,1,0.000000
261,5d7c994d5720533e15c3b1e9,5ebd1111514aab59896bcd1c,3.888604,3.886779,"{'actual_k': 3, 'was_impossible': False}",81,4,0.001825
280,5e7cf05bcfc8b713f5ac7db7,5ec3c09274f7660d73aa1229,3.742500,3.744383,"{'actual_k': 6, 'was_impossible': False}",40,7,0.001883


In [24]:
worst_predictions

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
175,5ea890e410426255a7aa9c16,5e9f4421a3258347b42f2e54,2.902508,3.463965,"{'actual_k': 4, 'was_impossible': False}",4,7,0.561457
117,5df49b32cc709107827fb3c7,5dee9b5042a8854bf6eabaaf,4.574167,3.937318,{'was_impossible': False},24,0,0.636849
173,5ed13bb076027d35905cc4ba,5ed3476576027d35905cca1d,4.593730,3.947873,"{'actual_k': 0, 'was_impossible': False}",1,1,0.645858
150,5e4195c3eab55d319938a82c,5eb1ac4510426255a7aaa024,4.593730,3.783393,{'was_impossible': False},5,0,0.810337
43,5d60098a653a331687083238,5eb1ac4510426255a7aaa024,4.720270,3.750366,{'was_impossible': False},191,0,0.969905


In [25]:
df_new = pd.merge(df_new,posts,on='post_id')
df_new = df_new[['user_id','post_id','title','category','post_type','rui','est','error']]
df_new

,user_id,post_id,title,category,post_type,rui,est,error
0,5d6d2bb87fa40e1417a49315,5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork,3.472770,3.400358,0.072412
1,5eb2cad810426255a7aaa072,5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork,3.472770,3.432859,0.039911
2,5deeef6142a8854bf6eabab9,5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork,3.472770,3.343169,0.129601
3,5e3563348d344822fed4d13a,5ecc1d4eeaff6b0c3a58a565,Faith in yourself,Photography,artwork,3.472770,3.465494,0.007276
4,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,Shiva Portrait,Drawings,artwork,3.472770,3.472770,0.000000
...,...,...,...,...,...,...,...,...
285,5d610ae1653a331687083239,5e78ae4ccfc8b713f5ac7cde,Sketch,Drawings,artwork,3.346230,3.665969,0.319739
286,5eb2cad810426255a7aaa072,5eca7601ec493f4a26558a62,Always do Right 👍🏻,Learning,skill,4.720270,4.677935,0.042335
287,5d60098a653a331687083238,5e7df068a3258347b42f2125,screw town,Photography,artwork,3.472770,3.451866,0.020904
288,5ecb979eeaff6b0c3a58a4f0,5ed0f0c576027d35905cc3c7,Designing Circular Microstrip Antenna,,project,4.574167,4.497683,0.076484


In [26]:
df_new[df_new['user_id']==df_new.user_id.value_counts().index[0]].sort_values(by='error',ascending=True)[:5]

,user_id,post_id,title,category,post_type,rui,est,error
37,5d610ae1653a331687083239,5e7f39a3a3258347b42f2151,The Meenamma,Drawings|Painting|Visual Arts|Artistic design|...,artwork,3.346230,3.338414,0.007816
28,5d610ae1653a331687083239,5ed0e31a76027d35905cc302,Designing Cmos circuit from Boolean expression...,,project,4.177897,4.189921,0.012024
136,5d610ae1653a331687083239,5ecc186aeaff6b0c3a58a53f,Beliefs ❣️,Visual Arts|Photography,artwork,3.346230,3.329634,0.016596
121,5d610ae1653a331687083239,5ea029e9a3258347b42f2e85,How will the economy be affected if Nations pr...,Economics|Economic Policies|Break even Point|F...,blog,3.762064,3.737141,0.024923
244,5d610ae1653a331687083239,5e2d516fc85ab714a7da66dd,'The Virtual ME',Fashion Design|Visual Arts|Graphic Design|Arti...,artwork,3.346230,3.371748,0.025518


In [27]:
df_new[df_new['user_id']==df_new.user_id.value_counts().index[-1]].sort_values(by='error',ascending=True)[:5]

,user_id,post_id,title,category,post_type,rui,est,error
222,5ebcf2f2514aab59896bcd00,5eab962e10426255a7aa9cb8,Platform for Guest Posting,Mixed Media,artwork,3.34623,3.507376,0.161145


In [28]:
test_df = pd.DataFrame(testset,columns = ['user_id','post_id','mixture'])
test_df

,user_id,post_id,mixture
0,5d6d2bb87fa40e1417a49315,5ecc1d4eeaff6b0c3a58a565,3.472770
1,5d60098a653a331687083238,5e7b1e65cfc8b713f5ac7d90,3.472770
2,5e26bb482a37d20505da2c10,5e2867f6025ae20886de205a,3.888604
3,5e5af599d701ab08af792b63,5ec7a8bdec493f4a26558846,3.615960
4,5d60098a653a331687083238,5eac350110426255a7aa9e92,3.888604
...,...,...,...
285,5e5af599d701ab08af792b63,5e8e13dca3258347b42f26f2,4.031793
286,5ea5ac4f10426255a7aa9b78,5e9ff423a3258347b42f2e72,4.158333
287,5e1ef04c2a37d20505da2b8b,5ec7aafbec493f4a26558857,3.742500
288,5e9c146ba3258347b42f2cbd,5e9b6b20a3258347b42f2c8d,4.031793


In [29]:
def get_n_recommend(user_id,n=5):
    test_results = pd.DataFrame(columns=['user_id','post_id','estimate'])
    for i in test_df.post_id.unique():
        tmp = pd.DataFrame([[user_id,i,algo.predict(user_id,i)[3]]] , columns=['user_id','post_id','estimate'])
        test_results = test_results.append(tmp,ignore_index=True)
    test_results = pd.merge(test_results,posts,on = 'post_id')
    return test_results.sort_values(by='estimate',ascending=False).reset_index(drop=True)[:n]

In [30]:
get_n_recommend(test_df.loc[0,'user_id'],10)

,user_id,post_id,estimate,title,category,post_type
0,5d6d2bb87fa40e1417a49315,5eaed2f210426255a7aa9eef,4.749437,Keen learner.,Photography,skill
1,5d6d2bb87fa40e1417a49315,5e9db770a3258347b42f2dcf,4.727005,E s c a p e,Photography,skill
2,5d6d2bb87fa40e1417a49315,5ed3476576027d35905cca1d,4.723314,AWS CLI Setup in Mac,Technology,skill
3,5d6d2bb87fa40e1417a49315,5e95ed0aa3258347b42f2a0e,4.712212,Lovely explosion box!,Craft work,skill
4,5d6d2bb87fa40e1417a49315,5ea7cd9610426255a7aa9bd2,4.712212,Guest Posting at Free of Cost,Business,skill
5,5d6d2bb87fa40e1417a49315,5ec54fc9f2781131cc7e50f5,4.698072,Future Communication Predictions,Science;Technology,skill
6,5d6d2bb87fa40e1417a49315,5ec573a3f2781131cc7e51b8,4.689692,My First Animated Post.,Art; Science,skill
7,5d6d2bb87fa40e1417a49315,5ebd5b46514aab59896bcd5a,4.684011,Quick Sketch of Gangster Skull.,Sketch Video,skill
8,5d6d2bb87fa40e1417a49315,5e7de250a3258347b42f210a,4.681265,Top 10 big benefits of having a digital market...,Digital Marketing,skill
9,5d6d2bb87fa40e1417a49315,5ed13d2876027d35905cc4c2,4.659350,Launching an EC2 instance.,Technology,skill
